### Define Imports

In [1]:
import create_template_model as CT

In [2]:
imports ="""
from itertools import product
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
from fbprophet import Prophet
import warnings
warnings.filterwarnings('ignore')
"""


### Define Class Name

In [3]:
class_name = "PROPHET"


###  Define Parameters for model

In [4]:

param = "param"

input_params = f"""
    self.parameters_list = kwargs.get('parameters_list',None)
"""

init_function = f"""
def __init__(self,**kwargs):

    #input parameters

    self.train,self.test = kwargs.get('train'),kwargs.get('test')       
    self.random_state = kwargs.get('random_state',1)
    self.target_dates = kwargs.get('target_dates')
    self.target_value = kwargs.get('target_value')
    self.target_items = kwargs.get('target_items')
    {input_params}
    self.n_periods = kwargs.get('n_periods',3)

    #output values
    self.fitted_values = pd.DataFrame()
    self.test_prediction = pd.DataFrame()
    self.unseen_prediction = pd.DataFrame()        
    self.apes = []
    self.run_model()
    del self.train,self.test
"""

### Fit method specific to Model

In [5]:
fit_method = f"""prophet_basic.fit(train_dataset)
"""

fit_function = f"""
def fit(self,data,exog_data,param):
        
    ###########
    #  Fit funtion will fit your train data and return model
    ###########
    return {fit_method}
"""

### Define Predict method

In [6]:
# run_script = None
predict_method = "prophet_basic.predict(future)"
predict_function =f"""
def predict(self,model,exog_data,start,n_periods):
    return {predict_method} 
""" 

### Fitted script

In [7]:
fitted_data = f"model.fittedvalues[param[6]+param[1]:]"
fitted_function =f"""
def fitted_data(self,model,param=None):
    return {fitted_data} 
""" 

### Additional Functions

In [8]:
additional_functions = """
def split_train_test(self,subset_df):
    return subset_df[:-3],subset_df[-3:]

def mean_absolute_percentage_error(self,y_true,y_pred):
    return np.mean(np.abs(np.subtract(y_true,y_pred)/y_true))*100  

def calculate_apes(self):
    for i,j in zip(self.test[self.target_value].values.flatten(),self.test_prediction.values.flatten()):            
        self.apes.append(self.mean_absolute_percentage_error(i,j))

def median_absolute_percentage_error_daily(self,y_true,y_pred):
    y_true=pd.Series(y_true)
    y_pred=pd.Series(y_pred)
    true_pred = pd.DataFrame(zip(y_true,y_pred),columns=['y_true','y_pred'])
    true_pred.drop(true_pred[true_pred['y_pred'] == 0].index, axis=0, inplace=True)
    true_pred.drop(true_pred[true_pred['y_true'] == 0].index, axis=0, inplace=True)
    return np.median(np.abs(np.subtract(true_pred.y_true,true_pred.y_pred)/true_pred.y_true))*100
    
def optimize_prophet(self,parameters_list,train_dataset,val_dataset,steps):  
    results=[]
    best_adj_mape=float('inf')
    for i in tqdm(parameters_list):
        forecast=pd.DataFrame()
        future=pd.DataFrame()
        
        prophet_basic = Prophet(growth='linear',daily_seasonality=False,weekly_seasonality=True,yearly_seasonality=True,holidays_prior_scale=10,n_changepoints=i[0],changepoint_prior_scale=i[1])
#        prophet_basic = Prophet(growth='linear',daily_seasonality=False,weekly_seasonality=True,yearly_seasonality=True,n_changepoints=20,changepoint_prior_scale=0.3)
        prophet_basic.add_regressor('is_extended')
        prophet_basic.add_regressor('Is_Month_End')
        prophet_basic.add_regressor('LastButOneDay')
        prophet_basic.add_regressor('LastSecDay')
#        prophet_basic.add_regressor('6th_Day')
        prophet_basic.add_regressor('13th_Day')
#        prophet_basic.add_regressor('20th_Day')
#        prophet_basic.add_regressor('27th_Day')
        prophet_basic.add_regressor('31st_Day')
        prophet_basic.add_country_holidays(country_name='US')
#        prophet_basic.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        prophet_basic.fit(train_dataset)
        
        future= prophet_basic.make_future_dataframe(periods=len(val_dataset))
        x=train_dataset.append(val_dataset)
        future['is_extended'] =pd.Series(x['is_extended'].values)
        future['Is_Month_End'] =pd.Series(x['Is_Month_End'].values)
        future['LastButOneDay'] =pd.Series(x['LastButOneDay'].values)
        future['LastSecDay'] =pd.Series(x['LastSecDay'].values)
#        future['6th_Day'] =pd.Series(x['6th_Day'].values)
        future['13th_Day'] =pd.Series(x['13th_Day'].values)
#        future['20th_Day'] =pd.Series(x['20th_Day'].values)
#        future['27th_Day'] =pd.Series(x['27th_Day'].values)
        future['31st_Day'] =pd.Series(x['31st_Day'].values)
        forecast=prophet_basic.predict(future)
        
        y_true=np.array(list(train_dataset['y']))
        y_pred=np.array(list(forecast.yhat[:-steps]))
        val_predicted=np.array(list(forecast.yhat[-steps:]))
        train_mape=round((median_absolute_percentage_error_daily(y_true[-365:],y_pred[-365:])),2)
        val_mape=round((median_absolute_percentage_error_daily(val_dataset["y"],val_predicted)),2)
        adj_mape = train_mape*len(y_true)/(len(y_true)+len(val_dataset))+val_mape*len(val_dataset)/(len(y_true)+len(val_dataset))
        
        if adj_mape <= best_adj_mape:
            best_adj_mape=adj_mape
            best_model = prophet_basic
            
        results.append([i,train_mape,val_mape,adj_mape])
        
    result_table=pd.DataFrame(results,columns=['parameters','train_mape','val_mape','adj_mape'])
    result_table=result_table.sort_values(by='adj_mape',ascending=True).reset_index(drop=True)
    return result_table, best_model
"""

### Running Models and storing results

In [9]:
run_script = f"""
def run_model(self):
#    train_set,val_set,test_set,be_name=train[["total_charge","is_extended","Is_Month_End","LastButOneDay","LastSecDay"]],val[["total_charge","is_extended","Is_Month_End","LastButOneDay","LastSecDay"]],test[["total_charge","is_extended","Is_Month_End","LastButOneDay","LastSecDay"]],be_name
    train_dataset= pd.DataFrame()
    val_dataset= pd.DataFrame()
    train_set=train_set.reset_index()
    val_set=val_set.reset_index()
    train_dataset['ds'] = train_set["posted_date"]
    train_dataset['y']=train_set["total_charge"]
    train_dataset['is_extended']=train_set["is_extended"]
    train_dataset['Is_Month_End']=train_set["Is_Month_End"]
    train_dataset['LastButOneDay']=train_set["LastButOneDay"]
    train_dataset['LastSecDay']=train_set["LastSecDay"]
#    train_dataset['6th_Day']=train_set["6th_Day"]
    train_dataset['13th_Day']=train_set["13th_Day"]
#    train_dataset['20th_Day']=train_set["20th_Day"]
#    train_dataset['27th_Day']=train_set["27th_Day"]
    train_dataset['31st_Day']=train_set["31st_Day"]
    val_dataset['ds'] = val_set["posted_date"]
    val_dataset['y']=val_set["total_charge"]
    val_dataset['is_extended']=val_set["is_extended"]
    val_dataset['Is_Month_End']=val_set["Is_Month_End"]
    val_dataset['LastButOneDay']=val_set["LastButOneDay"]
    val_dataset['LastSecDay']=val_set["LastSecDay"]
#    val_dataset['6th_Day']=val_set["6th_Day"]
    val_dataset['13th_Day']=val_set["13th_Day"]
#    val_dataset['20th_Day']=val_set["20th_Day"]
#    val_dataset['27th_Day']=val_set["27th_Day"]
    val_dataset['31st_Day']=val_set["31st_Day"]
    steps = len(val_set)
      
    cp=(.001,.005,0.01,0.05,0.1,0.3)
    ncp=(30,40)
    
    parameters=product(ncp,cp)
    parameters_list=list(parameters)
    result_table, best_model = optimize_prophet(parameters_list,train_dataset,val_dataset,steps)
    future2= best_model.make_future_dataframe(periods=steps)
    x=train_dataset.append(val_dataset)
    future2['is_extended'] =pd.Series(x['is_extended'].values)
    future2['Is_Month_End'] =pd.Series(x['Is_Month_End'].values)
    future2['LastButOneDay'] =pd.Series(x['LastButOneDay'].values)
    future2['LastSecDay'] =pd.Series(x['LastSecDay'].values)
#    future2['6th_Day'] =pd.Series(x['6th_Day'].values)
    future2['13th_Day'] =pd.Series(x['13th_Day'].values)
#    future2['20th_Day'] =pd.Series(x['20th_Day'].values)
#    future2['27th_Day'] =pd.Series(x['27th_Day'].values)
    future2['31st_Day'] =pd.Series(x['31st_Day'].values)
    forcast_val=best_model.predict(future2).yhat[-steps:]
    
    overall_train=pd.concat([train_set,val_set])
    overall_train['ds'] = overall_train.posted_date
    overall_train['y'] = overall_train["total_charge"]
    fitted_val_list=[]

    c=1
    for ncp,cp in result_table.parameters:
        try:
            if c > 3:
                break
            prophet_basic1 = Prophet(growth='linear',daily_seasonality=False,weekly_seasonality=True,yearly_seasonality=True,holidays_prior_scale=10,n_changepoints=ncp,changepoint_prior_scale=cp)
#            prophet_basic1.add_seasonality(name='monthly', period=30.5, fourier_order=5)
            prophet_basic1.add_regressor('is_extended')
            prophet_basic1.add_regressor('Is_Month_End')
            prophet_basic1.add_regressor('LastButOneDay')
            prophet_basic1.add_regressor('LastSecDay')
#            prophet_basic1.add_regressor('6th_Day')
            prophet_basic1.add_regressor('13th_Day')
#            prophet_basic1.add_regressor('20th_Day')
#            prophet_basic1.add_regressor('27th_Day')
            prophet_basic1.add_regressor('31st_Day')
            prophet_basic1.add_country_holidays(country_name='US')
            prophet_basic1.fit(overall_train)
            #if prophet_basic1.aic < res_aic_avg:
            future1= prophet_basic1.make_future_dataframe(periods=len(test_set))
            x=overall_train.append(test_set)
            future1['is_extended'] =pd.Series(x['is_extended'].values)
            future1['Is_Month_End'] =pd.Series(x['Is_Month_End'].values)
            future1['LastButOneDay'] =pd.Series(x['LastButOneDay'].values)
            future1['LastSecDay'] =pd.Series(x['LastSecDay'].values)
#            future1['6th_Day'] =pd.Series(x['6th_Day'].values)
            future1['13th_Day'] =pd.Series(x['13th_Day'].values)
#            future1['20th_Day'] =pd.Series(x['20th_Day'].values)
#            future1['27th_Day'] =pd.Series(x['27th_Day'].values)
            future1['31st_Day'] =pd.Series(x['31st_Day'].values)
            forecast=prophet_basic1.predict(future1).yhat[-(len(test_set)):]
            c=c+1
            get_list=[]
            for i in range(len(forecast)):
                get_list.append(forecast.iloc[i])
            fitted_val_list.append(get_list)

        except:
            continue
        
    fitted_val=pd.DataFrame(fitted_val_list,columns=[x for x in range(1,len(test_set)+1)])

    fitted_mean=[]
    for i in range(1,len(test_set)+1):
        fitted_mean.append(fitted_val[i].mean())
        
    test_set1=np.array(list(test_set["total_charge"]))
    test_results=round(median_absolute_percentage_error_daily(test_set1,fitted_mean),2)
"""

In [10]:
template_dict = dict(imports=imports,class_name=class_name,init_function=init_function,
                         fit_function=fit_function,fitted_function=fitted_function,predict_function=predict_function,
                        additional_functions=additional_functions,run_script=run_script)

In [11]:
CT.render_template(template_dict=template_dict)

In [12]:
print(CT.get_ouput())

#Import files


from itertools import product
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
from fbprophet import Prophet
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

#Model class
class PROPHET:

  

    def __init__(self,**kwargs):

        #input parameters

        self.train,self.test = kwargs.get('train'),kwargs.get('test')       
        self.random_state = kwargs.get('random_state',1)
        self.target_dates = kwargs.get('target_dates')
        self.target_value = kwargs.get('target_value')
        self.target_items = kwargs.get('target_items')
        
        self.parameters_list = kwargs.get('parameters_list',None)

        self.n_periods = kwargs.get('n_periods',3)

        #output values
        self.fitted_values = pd.DataFrame()
        self.test_prediction = pd.DataFrame()
        self.unseen_prediction = pd.DataFrame()        
        self.apes = []
        self.run_model()
        del self.train,s

In [13]:
class_name

'PROPHET'

In [1]:
from model.TBATS_MODEL import TBATS_CLASS
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.options.mode.chained_assignment = None

In [2]:
import json

with open('model_params.json') as f:
  config = json.load(f)

with open('data-config.json') as f1:
  data_config = json.load(f1)

In [3]:
config.keys()
data_config.keys()

dict_keys(['not_include_be_list', 'overall_be', 'aggregate_choice', 'target_file', 'target_dates', 'target_value', 'target_item', 'other_target_value', 'data_length', 'n_periods', 'minor_holidays', 'major_holidays', 'processed_data_path', 'wf_res_path', 'best_model_path'])

In [4]:
import os
import pickle
import pandas as pd

target_dates = 'posted_date'
target_values = 'total_charge'
target_items = 'billing_entity'
#transformed_data_path = 
#r'C:\Users\CK073783\OneDrive - Cerner Corporation\Desktop\Projects\Revenue Forecasting\Versions\version 2\data\processed_data\All Adventist W\\'

#data_files = os.listdir(transformed_data_path)
#model_output_path = r'C:\Users\CK073783\OneDrive - Cerner Corporation\Desktop\Projects\Revenue Forecasting\Versions\version 2\data\model_output\\'

def filter_data(ti):
    subset_df = preprocessed_data[preprocessed_data[target_items]==ti].reset_index()
    subset_df[target_dates] = pd.to_datetime(subset_df[target_dates])
    subset_df = subset_df.sort_values(target_dates)#.reset_index()
    subset_df=subset_df.drop([target_items],axis=1)
    subset_df.set_index(target_dates,inplace=True)
#    print("subset_df",subset_df.head(5))
    return subset_df

#filename = data_files[0]
filehandler = open(data_config["processed_data_path"], 'rb') 
data_object = pickle.load(filehandler)
data_date = data_object.summary_df.end_date.max().month_name() +'-'+str(data_object.summary_df.end_date.max().year)

preprocessed_data  = data_object.transform_data#[data_object.transform_data[target_items].isin(filter_data(data_object.summary_df))]
BE = 'All Adventist W'
subset_df=filter_data(BE)
#subset_df = filtered_data[filtered_data[target_items]==BE].reset_index(drop=True)
#subset_df[target_dates] = pd.to_datetime(subset_df[target_dates]).copy()
#subset_df.set_index('posted_date',inplace=True)
# models = Univariate(BE,data = subset_df[target_value],target_items= target_items,target_value=target_value,target_dates=target_dates)
subset_df.head()

,total_charge,footfall,minor_holiday,major_holiday,observed_holiday,is_extended,count_weekend
posted_date,,,,,,,
2015-05-31,8.659224e+08,261407.0,0,0,0,0,1
2015-06-30,1.020063e+09,298243.0,0,0,0,0,8
2015-07-31,1.034977e+09,307918.0,0,0,0,0,8
2015-08-31,1.017733e+09,286794.0,0,0,0,0,10
2015-09-30,9.889783e+08,293472.0,0,0,0,0,8


In [7]:
# from itertools import product
# def split_train_test(subset_df):
#         return subset_df[:-3],subset_df[-3:]
# train,test = split_train_test(subset_df)
#p=range(0,2)
#d=range(0,1)
#q=range(0,2)
#P=range(0,2)
# D=range(0,1)
# Q=range(0,2)
# s=(3,6,12)

# parameters=product(p,d,q,P,D,Q,s)
# parameters_list=list(parameters)

use_arma_errors=[True,False]
use_box_cox=[True,False]
use_trend=[True,False]
use_damped_trend=[True,False]  

parameters=product(use_arma_errors,use_box_cox,use_trend,use_damped_trend)
parameters_list=list(parameters)
print(parameters_list)
def check(**kwargs):
    print(kwargs.keys())
data_config['parameters_list'] = parameters_list
data_config['train'] = train
data_config['test'] = test
data_config['ti'] = BE
check(**data_config)
# obj = TBATS_CLASS(**data_config)
#print("in model.py results",df.mape_res)

[(True, True, True, True), (True, True, True, False), (True, True, False, True), (True, True, False, False), (True, False, True, True), (True, False, True, False), (True, False, False, True), (True, False, False, False), (False, True, True, True), (False, True, True, False), (False, True, False, True), (False, True, False, False), (False, False, True, True), (False, False, True, False), (False, False, False, True), (False, False, False, False)]
dict_keys(['not_include_be_list', 'overall_be', 'aggregate_choice', 'target_file', 'target_dates', 'target_value', 'target_item', 'other_target_value', 'data_length', 'n_periods', 'minor_holidays', 'major_holidays', 'processed_data_path', 'wf_res_path', 'best_model_path', 'parameters_list', 'train', 'test', 'ti'])


In [ ]:
obj.apes

In [ ]:
obj.unseen_prediction

In [9]:
['data', 'target_items', 'target_value', 'other_target_value', 'target_dates', 'n_periods', 'set_no', 'wf', 'parameters_list', 'train', 'test', 'ti']

['data',
 'target_items',
 'target_value',
 'other_target_value',
 'target_dates',
 'n_periods',
 'set_no',
 'wf',
 'parameters_list',
 'train',
 'test',
 'ti']